# Datasheet Analyzer Setup

이 노트북은 Datasheet Analyzer를 Google Colab에서 실행하기 위한 설정을 제공합니다.

In [ ]:
# GPU 사용 가능 여부 확인
!nvidia-smi

# PyTorch가 GPU를 인식하는지 확인
import torch
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"현재 사용 중인 GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리 사용량: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")

In [ ]:
# Clone the repository
!git clone https://github.com/doyoung42/autodatasheet.git
%cd autodatasheet

In [ ]:
# Install dependencies
!pip install -r requirements.txt

# Install additional dependencies for Google Colab
!pip install streamlit-option-menu

In [ ]:
# Python 패키지 경로 설정
import sys
import os
sys.path.append(os.getcwd())

# 로깅 설정
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Streamlit 실행
import subprocess
import threading
import time

def run_streamlit():
    try:
        logger.info("Starting Streamlit server...")
        subprocess.run([
            "streamlit", "run",
            "src/app/streamlit_app.py",
            "--server.port=8501",
            "--server.address=0.0.0.0",
            "--browser.serverAddress=0.0.0.0",
            "--server.headless=true"
        ])
    except Exception as e:
        logger.error(f"Error running Streamlit: {str(e)}")

# 백그라운드에서 Streamlit 실행
thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()

# 서버 시작 대기
time.sleep(5)

# 포트 포워딩
from google.colab.output import eval_js
print("Streamlit URL:")
print(eval_js("google.colab.kernel.proxyPort(8501)"))

In [ ]:
# Streamlit 로그 확인을 위한 셀
import time
import subprocess

def check_streamlit_logs():
    """Streamlit 로그를 확인하는 함수"""
    try:
        # 먼저 로그 파일이 있는지 확인
        result = subprocess.run(['find', '/', '-name', 'streamlit.log', '2>/dev/null'], 
                              capture_output=True, text=True)
        
        if result.stdout.strip():
            log_path = result.stdout.strip().split('\n')[0]
            print(f"로그 파일을 찾았습니다: {log_path}")
            # 로그 파일 실시간 모니터링
            subprocess.run(['tail', '-f', log_path])
        else:
            print("streamlit.log 파일을 찾을 수 없습니다.")
            print("현재 실행 중인 Streamlit 프로세스 확인:")
            subprocess.run(['ps', 'aux', '|', 'grep', 'streamlit'])
            
            print("\n현재 디렉토리의 로그 파일 확인:")
            subprocess.run(['ls', '-la', '*.log'])
            
            print("\n/tmp 디렉토리의 로그 파일 확인:")
            subprocess.run(['ls', '-la', '/tmp/*.log'])
    except Exception as e:
        print(f"로그 확인 중 오류 발생: {str(e)}")

# 로그 확인 시작
print("Streamlit 로그 확인을 시작합니다...")
check_streamlit_logs()